In [1]:
import MetaTrader5 as mt5
from datetime import datetime
import keras
import pandas as pd 
import numpy as np

model = keras.models.load_model("LSTM_eurusd2.model")


Using TensorFlow backend.


In [2]:
mt5.initialize()
symbol = "EURUSD"   
lot = 0.1

rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_H1, 0, 100)
rates = pd.DataFrame(rates)
rates['time']=pd.to_datetime(rates['time'], unit='s')


In [3]:
X = rates.loc[-2:-1,['open','high','low','close', 'tick_volume']].values
Xpred = np.reshape(X, (X.shape[0], 1, X.shape[1]))
Yhat = model.predict(Xpred)
xhat = X[0,0]
yhat = Yhat[0,0]

ValueError: Error when checking input: expected bidirectional_2_input to have shape (1, 1) but got array with shape (1, 4)

In [80]:
def open_trade(action, symbol, lot, sl_points, tp_points, deviation):
    '''https://www.mql5.com/en/docs/integration/python_metatrader5/mt5ordersend_py
    '''
    # prepare the buy request structure
    symbol_info = mt5.symbol_info(symbol)

    if action == 'buy':
        trade_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
        slval = round(price - sl_points * 0.0001,5)
        tpval = round(price + tp_points * 0.0001,5)
    elif action =='sell':
        trade_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
        slval = round(price + sl_points * 0.0001,5)
        tpval = round(price - tp_points * 0.0001,5)
    point = mt5.symbol_info(symbol).point

    buy_request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": trade_type,
        "price": price,
        "sl": slval,
        "tp": tpval,
        "deviation": deviation,
        "magic": ordernum,
        "comment": "sent by python",
        "type_time": mt5.ORDER_TIME_GTC,  # good till cancelled
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    # send a trading request
    result = mt5.order_send(buy_request)        
    return result, buy_request 

In [81]:
tp_points = 20
sl_points = 20
deviation = 5
ordernum = 10001

if yhat >= xhat:
    action = "buy"
else:
    action = "sell"

#price = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M1, 0,1)[0][4]

if 
open_trade(action, symbol, lot, sl_points, tp_points, deviation)




(OrderSendResult(retcode=10009, deal=700407891, order=721011876, volume=0.1, price=1.16356, bid=1.16357, ask=1.16361, comment='Request executed', request_id=1, retcode_external=0, request=TradeRequest(action=1, magic=10001, order=0, symbol='EURUSD', volume=0.1, price=1.16356, stoplimit=0.0, sl=1.17356, tp=1.15356, deviation=10, type=1, type_filling=2, type_time=0, expiration=0, comment='sent by python', position=0, position_by=0)),
 {'action': 1,
  'symbol': 'EURUSD',
  'volume': 0.1,
  'type': 1,
  'price': 1.16356,
  'sl': 1.17356,
  'tp': 1.15356,
  'deviation': 10,
  'magic': 10001,
  'comment': 'sent by python',
  'type_time': 0,
  'type_filling': 2})

In [16]:
import MetaTrader5 as mt5
mt5.initialize()
pos = mt5.positions_get("EURUSD")

In [20]:
positions=mt5.positions_get(symbol="EURUSD")
positions

()